In [21]:
import torch
import numpy as np
import os
#import torch.neuron
import blink.main_dense as main_dense
import argparse
import json 
import time
import torch_neuron

In [22]:
print(torch.__version__)
print(torch_neuron.__version__)

1.10.1+cu102
1.10.1.2.1.7.0


In [4]:
device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
device

device(type='cuda')

In [5]:
#models_path = "../models/" # the path where you stored the BLINK models
models_path = "/home/ubuntu/BLINK/torchserve_sandbox/model_support"#'/opt/models/model-support/'#"model_store/" # the path where you stored the BLINK models

config = {
    "test_entities": None,
    "test_mentions": None,
    "interactive": False,
    "top_k": 10,
    "biencoder_model": models_path+"/biencoder_wiki_large.bin",
    "biencoder_config": models_path+"/biencoder_wiki_large.json",
    "entity_catalogue": models_path+"/entity.jsonl",
    "entity_encoding": models_path+"/all_entities_large.t7",
    "crossencoder_model": models_path+"/crossencoder_wiki_large.bin",
    "crossencoder_config": models_path+"/crossencoder_wiki_large.json",
    "fast": True, # set this to be true if speed is a concern
    "output_path": "logs/" # logging directory
}

args = argparse.Namespace(**config)

data_to_link = [ {
                    "id": 0,
                    "label": "unknown",
                    "label_id": -1,
                    "context_left": "".lower(),
                    "mention": "Shakespeare".lower(),
                    "context_right": "'s account of the Roman general Julius Caesar's murder by his friend Brutus is a meditation on duty.".lower(),
                },
                {
                    "id": 1,
                    "label": "unknown",
                    "label_id": -1,
                    "context_left": "Shakespeare's account of the Roman general".lower(),
                    "mention": "Julius Caesar".lower(),
                    "context_right": "'s murder by his friend Brutus is a meditation on duty.".lower(),
                }
                ]

In [6]:
models = main_dense.load_models(args, logger=None)

biencoder, biencoder_params, crossencoder, crossencoder_params, candidate_encoding, title2id, id2title, id2text, wikipedia_id2local_id, faiss_indexer = models


cuda


In [7]:
cand_encs = torch.load('sample_input/cand_encs.pt')
text_vecs = torch.load('sample_input/text_vecs.pt')
inputs = [text_vecs, cand_encs]

In [8]:
random_negs = True
model = biencoder

In [9]:
x=model(*inputs)
x

tensor([[71.5985, 72.7923, 82.1700, 73.7910, 67.3760],
        [72.4845, 84.5333, 72.7498, 68.8741, 63.2799]], device='cuda:0',
       grad_fn=<MmBackward0>)

In [10]:
start = time.time()

_, _, _, _, _, predictions, scores, mention_found= main_dense.run(args, None, *models, test_data=data_to_link, REL_filter=True)
print(predictions)

end = time.time()
print(end - start)

100%|██████████| 1/1 [00:00<00:00, 33.72it/s][['William Shakespeare', 'Romeo and Juliet'], ['Julius Caesar', 'William Shakespeare']]
0.09079742431640625



In [11]:
module = torch.jit.trace(model, example_inputs=inputs)
module.save("blink_entity_linking.pt")

In [12]:
module_loaded=torch.jit.load("blink_entity_linking.pt")

In [13]:
x=module_loaded(*inputs)
x

tensor([[71.5985, 72.7923, 82.1700, 73.7910, 67.3760],
        [72.4845, 84.5333, 72.7498, 68.8741, 63.2799]], device='cuda:0',
       grad_fn=<MmBackward0>)

In [14]:
models_jit = module_loaded, biencoder_params, crossencoder, crossencoder_params, candidate_encoding, title2id, id2title, id2text, wikipedia_id2local_id, faiss_indexer 

In [15]:
start = time.time()

_, _, _, _, _, predictions, scores, mention_found= main_dense.run(args, None, *models_jit, test_data=data_to_link, REL_filter=True)
print(predictions)

end = time.time()
print(end - start)

100%|██████████| 1/1 [00:00<00:00,  2.08it/s][['William Shakespeare', 'Romeo and Juliet'], ['Julius Caesar', 'William Shakespeare']]
0.5425407886505127



In [16]:
# user newer neuron version, driver version etc. if cannot compiled
model = biencoder
compiler_options = "-O2"
optimize = "aggressive"

module_neuron = torch_neuron.trace(model, example_inputs=inputs, compiler_args=compiler_options, optimize = optimize)
#model_neuron = torch.neuron.script(model, compiler_args=compiler_options, optimize = optimize)

INFO:Neuron:There are 3 ops of 1 different types in the TorchScript that are not compiled by neuron-cc: aten::embedding, (For more information see https://github.com/aws/aws-neuron-sdk/blob/master/release-notes/neuron-cc-ops/neuron-cc-ops-pytorch.md)
INFO:Neuron:Number of arithmetic operators (pre-compilation) before = 1205, fused = 1186, percent fused = 98.42%
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:Neuron:Compiler args type is <class 'str'> value is -O2
INFO:Neuron:Compiling function _NeuronGraph$1272 with neuron-cc
INFO:Neuron:Compiling with command line: '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/bin/neuron-cc compile /tmp/tmpf1lxpjac/graph_def.pb --framework TENSORFLOW --pipeline compile SaveTemps --output /tmp/tmpf1lxpjac/graph_def.neff --io-config {"inputs": {"0:0": [[2, 32, 1024], "float32"], "1:0": [[2, 1, 1, 32], "float32"], "2:0": [[5, 1024], "float32"]}, "outputs": ["MatMul_192:0"]} -O2 --verbose 35'
INFO:Neuro

In [17]:
module_neuron.save("blink_enlity_linking_neuron.pt")

In [18]:
model(*inputs)

tensor([[71.5985, 72.7923, 82.1700, 73.7910, 67.3760],
        [72.4845, 84.5333, 72.7498, 68.8741, 63.2799]], device='cuda:0',
       grad_fn=<MmBackward0>)

In [19]:
module_neuron

AwsNeuronGraphModule(
  original_name=AwsNeuronGraphModule
  (_NeuronGraph#55): NeuronModuleV2(original_name=NeuronModuleV2)
)

In [20]:
module_neuron(*inputs)

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/torch_neuron/decorators.py(373): forward
/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/torch/nn/modules/module.py(1090): _slow_forward
/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/torch/nn/modules/module.py(1102): _call_impl
/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/torch_neuron/graph.py(546): __call__
/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/torch_neuron/graph.py(205): run_op
/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/torch_neuron/graph.py(194): __call__
/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/torch_neuron/convert.py(216): forward
/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-p

In [20]:
module_loaded_neuron=torch.jit.load("working_neuron/blink_enlity_linking_neuron.pt")

In [24]:
module_loaded_neuron

RecursiveScriptModule(
  original_name=AwsNeuronGraphModule
  (_NeuronGraph#55): RecursiveScriptModule(original_name=NeuronModule)
)

In [21]:
module_loaded_neuron.forward(*inputs)

tensor([[71.5984, 72.7923, 82.1700, 73.7910, 67.3760],
        [72.4845, 84.5333, 72.7498, 68.8741, 63.2799]])

In [22]:
models_jit_neuron = module_loaded_neuron, biencoder_params, crossencoder, crossencoder_params, candidate_encoding, title2id, id2title, id2text, wikipedia_id2local_id, faiss_indexer 

In [23]:
start = time.time()

_, _, _, _, _, predictions, scores, = main_dense.run(args, None, *models_jit_neuron, test_data=data_to_link, REL_filter=True)
print(predictions)

end = time.time()
print(end - start)

  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: forward() is missing value for argument 'argument_2'. Declaration: forward(__torch__.torch_neuron.convert.AwsNeuronGraphModule self, Tensor argument_1, Tensor argument_2) -> (Tensor)